In [1]:
from analysis import *

models=['o1-preview', 'o1-mini', 'gpt-4-0125-preview', 'gpt-4o', 'gpt-4o-mini', 'claude-3-5-sonnet-20240620', 
        'gemini-1.5-pro', 'llama-3.1-405b', 'llama-3.1-70b', 'llama-3.1-8b', 'qwen2.5-72b-instruct', 'qwen2.5-32b-instruct','qwen2.5-14b-instruct']
model_name = ['o1-preview','o1-mini','gpt-4','gpt-4o', 'gpt-4o-mini', 'claude-3.5-sonnet',  'gemini-1.5-pro', 
              'llama-3.1-405b', 'llama-3.1-70b', 'llama-3.1-8b', 'qwen2.5-72b', 'qwen2.5-32b','qwen2.5-14b']
[results, acc_2back, dec1, dec2, rew1, rew2, rew_avg, rules1, tests1, options1, feedbacks1,rules2, tests2, options2, feedbacks2, metadec1, 
 metadec2, metarew1, metarew2, wpt_today1, wpt_device1, wpt_pred1, wpt_tomorrow1, wpt_today2, wpt_device2, wpt_pred2, wpt_tomorrow2, selections1_session1,
   gains1_session1, losss1_session1, overages1_session1, selections2_session1, gains_session1, losss_session1, overages_session1, selections1_session2, 
   gains1_session2, losss1_session2, overages1_session2, selections2_session2, gains_session2, losss_session2, overages_session2] = read_outcomes(models)

In [ ]:
# oddball paradigm
filepath = '../results/oddball_human_evaluation.jsonl'
mean_oddball, all_oddball = oddball_scoring(filepath)
diagram_scores(mean_oddball, "Surprisal level", "Oddball paradigm", model_name)
plot_mmn_waveforms(all_oddball, model_name)

In [ ]:
# 2-back
diagram_scores(acc_2back, 'Scores', '2-BACK', model_name)

In [ ]:
# prt
errorprt = visualize_prlt_decision(dec1, dec2,'Ratio of left arm','Probability Reversal Learning Task', labels= model_name, window_size=3)
errs,prt_scores = np.zeros(13), np.zeros(13)
for i in range(13):
    err = (1-(errorprt[i])/0.832)*100
    errs[i] = err
    prt_scores[i] = (err+ rew_avg[i])/2
diagram_scores(errs,'Consistency with true probability','Probabilistic Reversal Learning Task',model_name)
diagram_scores(prt_scores,'Overall scores','Probabilistic Reversal Learning Task',model_name)

In [ ]:
# wcst
wcst_score = []
for i in range(13):
    wcst_score.append((np.sum(feedbacks1[i]) + np.sum(feedbacks2[i]) )/ 102 * 50)

diagram_scores(wcst_score, "Overall scores", "Wisconsin Card Sorting Test", model_name)

shape_acc_1, color_acc_1, number_acc_1,shape_acc_2, color_acc_2, number_acc_2 = [],[],[],[],[],[]
for i in range(13):
    shape_acc_1.append((np.sum(feedbacks1[i][0:18]) + np.sum(feedbacks2[i][0:18]) )/ 18 * 50)
    color_acc_1.append((np.sum(feedbacks1[i][18:36]) + np.sum(feedbacks2[i][18:36]) )/ 18 * 50)
    number_acc_1.append((np.sum(feedbacks1[i][36:54]) + np.sum(feedbacks2[i][36:54]) )/ 18 * 50)
    shape_acc_2.append((np.sum(feedbacks1[i][54:72]) + np.sum(feedbacks2[i][54:72]) )/ 18 * 50)
    color_acc_2.append((np.sum(feedbacks1[i][72:90]) + np.sum(feedbacks2[i][72:90]) )/ 18 * 50)
    number_acc_2.append((np.sum(feedbacks1[i][90:108]) + np.sum(feedbacks2[i][90:108]) )/ 18 * 50)

plot_accuracy_heatmap(shape_acc_1, color_acc_1, number_acc_1, shape_acc_2, color_acc_2, number_acc_2, model_name)

In [ ]:
# weather prediction task
estimated_pred_transitions1, estimated_real_transitions1, estimated_pred_transitions2, estimated_real_transitions2 = [],[], [],[]

for i in range(13):
    pred_transition1 = estimate_transition_probabilities(wpt_today1[i][80:100],wpt_device1[i][80:100],wpt_pred1[i][80:100])
    real_transition1 = estimate_transition_probabilities(wpt_today1[i][80:100],wpt_device1[i][80:100],wpt_tomorrow1[i][80:100])
    pred_transition2 = estimate_transition_probabilities(wpt_today2[i][80:100],wpt_device2[i][80:100],wpt_pred2[i][80:100])
    real_transition2 = estimate_transition_probabilities(wpt_today2[i][80:100],wpt_device2[i][80:100],wpt_tomorrow2[i][80:100])
    estimated_pred_transitions1.append(pred_transition1)
    estimated_real_transitions1.append(real_transition1)
    estimated_pred_transitions2.append(pred_transition2)
    estimated_real_transitions2.append(real_transition2)

diffs1, diffs2 = [], []
for i in range(13):
    print(model_name[i], "session1: ")
    diff1 = visualize_wpt_heatmap(estimated_pred_transitions1[i],estimated_real_transitions1[i])
    print(model_name[i], "session2: ")
    diff2 = visualize_wpt_heatmap(estimated_pred_transitions2[i],estimated_real_transitions2[i])
    diffs1.append(diff1)
    diffs2.append(diff2)

scores = []
for i in range(13):
    score1 = score_diff(estimated_real_transitions1[i], diffs1[i])
    score2 = score_diff(estimated_real_transitions2[i], diffs2[i])
    score = log_weighted_average(score1, score2)
    scores.append(score)

diagram_scores(scores, "Scores", "Weather prediction task", model_name)

In [ ]:
#DCIGT
overages_mean = []
for i in range(13):
    meanoverages=[]
    for j in range(100):
        overage_mean = (overages_session1[i][j] + overages_session2[i][j]) / 2
        meanoverages.append(overage_mean)
    overages_mean.append(meanoverages)

visualize_igt(overages_mean, "DCIGT", model_name)

switches1, scores_dcigt1, switches2, scores_dcigt2 = [], [], [], []# short term
overages_final1,  overages_final2 = [], []# long term

for i in range(13):
    switch_num1, score_dcigt1 = beneficial_switching(selections1_session1[i], selections2_session1[i], losss1_session1[i])
    switch_num2, score_dcigt2 = beneficial_switching(selections1_session2[i], selections2_session2[i], losss1_session2[i])
    switches1.append(switch_num1)
    scores_dcigt1.append(score_dcigt1)
    switches2.append(switch_num2)
    scores_dcigt2.append(score_dcigt2)
    overages_final1.append(overages_session1[i][99])
    overages_final2.append(overages_session2[i][99])

normalized1 = min_max_norm(overages_final1)*100
normalized2 = min_max_norm(overages_final2)*100
normalized3 = min_max_norm(scores_dcigt1)*100
normalized4 = min_max_norm(scores_dcigt2)*100

overall_scores_dcigt,dc11,dc22 = [],[],[]

for i in range(13):
    #overall_score = log_weighted_average(log_weighted_average(normalized1[i], normalized2[i]), log_weighted_average(normalized3[i], normalized4[i]))
    dcscore1 = (normalized1[i] + normalized2[i])/2
    dcscore2 = (normalized3[i] + normalized4[i])/2
    overall_score = (normalized1[i] + normalized2[i])* 0.25 + (normalized3[i]+ normalized4[i]) * 0.25
    overall_scores_dcigt.append(overall_score)
    dc11.append(dcscore1)
    dc22.append(dcscore2)


diagram_scores(overall_scores_dcigt,"Scores","DCIGT",model_name)

In [ ]:
# meta-prlt
meanmetaprtrew=[]
for i in range(13):
    meanrewmeta = []
    for j in range(60):
        meanrewmeta.append((metarew1[i][j] + metarew2[i][j])/2)
    meanmetaprtrew.append(meanrewmeta)

visualize_metaprt(np.array(metarew1),1,model_name)
visualize_metaprt(np.array(metarew2),2,model_name)

In [ ]:
#overall
overall(mean_oddball, acc_2back, prt_scores, wcst_score, scores, overall_scores_dcigt, model_name)